In [1]:
from pykrx import stock
import mpl_finance

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import arrow

/Users/minseop/.pyenv/versions/3.8.5/envs/env_stock_bot/lib/python3.8/site-packages/mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [2]:
# constant variables
TRAINING_DAYS = 180
DAYS_FOR_PREDICTION = 7
DATE_TICK_RANGE = int(TRAINING_DAYS/9)

In [3]:
def get_stock_info_dataframe(end_date, start_date, ticker):
    df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)
    df = df.rename(columns={
                      '시가': 'open',
                      '고가': 'high',
                      '저가': 'low',
                      '종가': 'close',
                      '거래량': 'volume'})
    
    open_next_day = []
    high_next_day = []
    low_next_day = []
    close_next_day = []
    volume_next_day = []
    
    open_days_ago = []
    high_days_ago = []
    low_days_ago = []
    close_days_ago = []
    volume_days_ago = []

    for ago_days in range(0, DAYS_FOR_PREDICTION):
        open_days_ago.append([])
        high_days_ago.append([])
        low_days_ago.append([])
        close_days_ago.append([])
        volume_days_ago.append([])
    
    
    for i in range(DAYS_FOR_PREDICTION-1, len(df)-1):
        open_next_day.append(df.iloc[i+1]['open'])
        high_next_day.append(df.iloc[i+1]['high'])
        low_next_day.append(df.iloc[i+1]['low'])
        close_next_day.append(df.iloc[i+1]['close'])
        volume_next_day.append(df.iloc[i+1]['volume'])
        
        for ago_days in range(1, DAYS_FOR_PREDICTION):
            open_days_ago[ago_days].append(df.iloc[i-ago_days]['open'])
            high_days_ago[ago_days].append(df.iloc[i-ago_days]['high'])
            low_days_ago[ago_days].append(df.iloc[i-ago_days]['low'])
            close_days_ago[ago_days].append(df.iloc[i-ago_days]['close'])
            volume_days_ago[ago_days].append(df.iloc[i-ago_days]['volume'])

    df = df.iloc[DAYS_FOR_PREDICTION-1:len(df)-1,]
    for ago_days in range(1, DAYS_FOR_PREDICTION):
        df[f'open_{ago_days}day_ago'] = open_days_ago[ago_days]
        df[f'high_{ago_days}day_ago'] = high_days_ago[ago_days]
        df[f'low_{ago_days}day_ago'] = low_days_ago[ago_days]
        df[f'close_{ago_days}day_ago'] = close_days_ago[ago_days]
        df[f'volume_{ago_days}day_ago'] = volume_days_ago[ago_days]
    
    df['open_next_day'] = open_next_day
    df['high_next_day'] = high_next_day
    df['low_next_day'] = low_next_day
    df['close_next_day'] = close_next_day
    df['volume_next_day'] = volume_next_day

    return df

In [4]:
# ticker=005930: Samsung Electronics
# ticker=000660: Hynix
end_date = arrow.utcnow().shift(hours=9).format('YYYYMMDD')
start_date = arrow.get(end_date).shift(days=-TRAINING_DAYS).format('YYYYMMDD')

df_samsung = get_stock_info_dataframe(end_date, start_date, '005930')

df_samsung

,open,high,low,close,volume,open_1day_ago,high_1day_ago,low_1day_ago,close_1day_ago,volume_1day_ago,...,open_6day_ago,high_6day_ago,low_6day_ago,close_6day_ago,volume_6day_ago,open_next_day,high_next_day,low_next_day,close_next_day,volume_next_day
날짜,,,,,,,,,,,,,,,,,,,,,
2021-03-04,82600,83200,82200,82400,24348331,83500,84000,82800,84000,19882132,...,81200,82900,81100,82000,20587314,81100,82600,81100,82100,20508971
2021-03-05,81100,82600,81100,82100,20508971,82600,83200,82200,82400,24348331,...,81800,83600,81300,82000,26807651,82900,83000,81600,82000,17641256
2021-03-08,82900,83000,81600,82000,17641256,81100,82600,81100,82100,20508971,...,84000,85400,83000,85300,34155986,81400,81900,80600,81400,25420764
2021-03-09,81400,81900,80600,81400,25420764,82900,83000,81600,82000,17641256,...,82800,83400,82000,82500,38520800,82400,82500,80700,80900,20038422
2021-03-10,82400,82500,80700,80900,20038422,81400,81900,80600,81400,25420764,...,85100,85300,83000,83600,33498180,81000,82500,81000,82000,23818297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-12,77100,78200,76900,77000,42365223,79600,79800,78500,78500,30241137,...,82200,83100,81800,82900,25642368,75800,76000,74100,74400,61270643
2021-08-13,75800,76000,74100,74400,61270643,77100,78200,76900,77000,42365223,...,83300,83300,82000,82100,18485469,74000,75100,74000,74200,30944847
2021-08-17,74000,75100,74000,74200,30944847,75800,76000,74100,74400,61270643,...,81900,82500,81300,81500,13342623,73900,74600,73100,73900,29192631


In [5]:
def draw_candle_chart(df_stock_info):
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(111)

    day_list = range(len(df_stock_info))
    name_list = []

    cnt = 0
    for day in df_stock_info.index:
        name_list.append(arrow.get(day).format('YYYY-MM-DD')) if cnt % DATE_TICK_RANGE == 0 else name_list.append('')
        cnt = cnt + 1

    ax.xaxis.set_major_locator(ticker.FixedLocator(day_list))
    ax.xaxis.set_major_formatter(ticker.FixedFormatter(name_list))

    mpl_finance.candlestick2_ohlc(ax, df_stock_info['open'], df_stock_info['high'],
                                  df_stock_info['low'], df_stock_info['close'], width=0.5, colorup='r', colordown='b')
    plt.show()